# Průzkum dat a jejich úpravy
Vytváření modelů si budeme ukazovat na volně dostupných datasetech.

Velmi často se stává, že surová data obsahují chyby, některá data chybí, data jsou ve špatném formátu apod.

Vždy než začnete s daty pracovat, je dobré jim porozumět a případně si upravit.

Na internetu je celá řada volně dostupných datových zdrojů, na kterých můžete zkoušet své schopnosti.
- https://archive.ics.uci.edu/ml/index.php
- https://www.kaggle.com/
- https://toolbox.google.com/datasetsearch
- github datasety

## Boston Housing Dataset

Soubor dat o bydlení je odvozen z informací o bydlení v oblasti Bostonu ve státě Massachusetts, které shromáždil americký úřad pro sčítání lidu.  

Údaje byly původně publikovány v článku od  Harrison, D. and Rubinfeld, D.L. `Hedonic prices and the demand for clean air', J. Environ. Economics & Management, vol.5, 81-102, 1978. 

Datová sada obsahuje informace o 506 různých domech v Bostonu.

Dataset features
* CRIM - míra kriminality na obyvatele podle měst
* ZN - podíl pozemků pro bydlení s rozlohou nad 25 000 m2.
* INDUS - podíl neobchodních ploch pro podnikání na město
* CHAS - dummy proměnná Charles River (1, pokud trakt hraničí s řekou; 0 jinak)
* NOX - koncentrace oxidů dusíku (částic na 10 milionů)
* RM - průměrný počet pokojů na byt
* AGE - podíl vlastnických bytů postavených před rokem 1940
* DIS - vážené vzdálenosti do pěti bostonských center zaměstnanosti
* RAD - index dostupnosti k radiálním dálnicím
* TAX - sazba daně z nemovitosti v plné hodnotě na 10 000 dolarů
* PTRATIO - poměr žáků a učitelů podle měst
* B - 1000(Bk - 0,63)^2, kde Bk je podíl černochů v jednotlivých městech.
* LSTAT - procento nižšího stavu obyvatelstva
* MEDV - mediánová hodnota obydlí obývaných vlastníky v 1000 USD

## Přečtení dat z CSV souboru

In [ ]:
import pandas as pd 

In [ ]:
data = pd.read_csv ("..\\dataset\\HousingData.csv")

Podíváme se na strukturu souboru.

In [ ]:
data.info()

## Základní charakteristiky data

Před tvorbou modelu je vhodné mít přehled o vstupních datech.
Tím lze předejít pozdějším problémům. Některé modely například vyžadují specifická data.

Zobrazení náhledu na data.

In [ ]:
data.head(10)

Základní statistika dat ve sloupcích se zobrazí pomocí funkce **describe**
- počet záznamů
- střední hodnota - průměr
- rozptyl
- minimum
- 25% percentil
- 50% percentil - medián
- 75% percentil
- maximum

U některých sloupců se výrazně liší průměr a medián - CRIM, ZN

U některých sloupců jsou průměr a medián podobné - RM

Dobře vidět to bude při zobrazení distribuce hodnot.

In [ ]:
data.describe()

Některé sloupce obsahují data NULL. Musíme se rozhodnout, jak tento problém vyřešit.
* Neúplné řádky lze z datové sady odstranit
* Problematické sloupce by neměly být vstupními parametry modelu
* Chybějící hodnoty by mohly být vyrobeny jako průměr, nuly, ...
*...

In [ ]:
data.isna().sum()

## Distribuce hodnot

Vizualizace rozložení dat ve sloupcích by mohla objevit zkreslené, abnormální hodnoty.

Zároveň některé statistické metody nemusí na atypicky rozložených datech fungovat správně.

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

Distribuci dat lze dobře pochopit z grafů.

Vytvoříme si graf, který bude kombinovat histogram s odhadem distribuční funkce.

Odhad distribuční funkce získáme pomocí knihovny seaborn a  kernel density estimate line.

Z grafů je vidět, že některé proměnné mají téměř normální rozdělení (RM), jiné ho mají skoro rovnoměrné (NOX). 

Některé proměnné mají velké zastoupení malých hodnot a vysoké hodnoty se v datasetu téměř nevyskytují (CRIM).

U některý proměnných je vidět, že jsou hodně zastoupeny maximální hodnoty (B, TAX).

In [ ]:
pos = 1
fig = plt.figure(figsize=(16,24))
for i in data.columns:
    ax = fig.add_subplot(7,2, pos)
    pos = pos + 1
    sns.histplot(data[i], ax=ax, kde=True)

Podobné informace lze vyčíst i boxplotu. Sice údaje o distribuci nejsou tak detailní, ale graf je kompaktní. Navíc z něj lze velmi pěkně vyčíst vychýlené hodnoty.

Proto se velmi často objevuje v odborných článcích, když na malém prostoru potřebujete představit data.

In [ ]:
plt.figure(figsize=(12, 6))
sns.boxplot(data=data, )

## Vztahy mezi proměnnými
V datasetech bývá velmi mnoho proměnných. Často se stává, že mezi nimi je nějaký vztah. Pokud se jedna proměnná změní, tak se pravděpodobně změní i jiná proměnná.

Tyto vztahy mohou a nemusí mít příčinnou souvislost. Někdy se může jednat i o náhodu. 

Proto je dobré zkusit tyto vztahy odhalit. 
* Odhalení závislostí – zjistíme, jestli změna jedné proměnné souvisí se změnou jiné (např. výška a hmotnost).
* Redundance – silně korelované proměnné často nesou stejnou informaci → při modelování není nutné mít obě.
* Predikce – pokud jedna proměnná silně souvisí s druhou, můžeme ji využít k předpovědi (např. věk ↔ příjem).
* Skryté vztahy – slabá nebo žádná korelace může znamenat, že závislost je nelineární nebo ji ovlivňují další faktory.

Existuje celá řada metod, jak závislosti odhalovat.

Začneme tím, že si vytvoříme grafy pro všechny kombinace dvou funkcí.

Lidský mozek je trénovaný na hledání vzorů. Může se stát, že nějaký vztah uvidíme na první pohled. 

Zpravidla hledáme takový tvar grafu, který zobrazuje nějakou matematickou křivku (přímka, parabola, hyperbola, apod.)

In [ ]:
sns.pairplot(data)

Z grafů je vidět, že existuje určitá přímá úměra mezi RM, LSTAT a MEDV.

* RM - průměrný počet pokojů na byt (vstupní proměnná)
* LSTAT - procentuálně nižší stav populace (vstupní proměnná)
* MDEV - mediánová hodnota obydlí obývaných vlastníky v 1000 USD (výstupní proměnná)

Vztahy mezi proměnnými jsme odhadovali podle oka. Ale lze to i dělat exaktně pomocí korelace.

Korelace nám ukazuje, jak silně a jakým směrem jsou dvě (nebo více) proměnné **lineárně** závislé. Pozor některé jevy nemusí mít lineární závislost, ale jinou. Pro jiný typ vztahů než je lineární korelační koeficient nebude fungovat.

Korelační koeficient (Pearsonův r):
* Hodnoty od -1 do 1
* r ≈ 1 → silná pozitivní lineární závislost
* r ≈ -1 → silná negativní lineární závislost
* r ≈ 0 → žádná lineární závislost (může být ale nelineární)

In [ ]:
corr=data.corr()
corr

Silná přímá nebo nepřímá korelace může naznačovat vztah mezi vstupními parametry.
To nám může pomoci při volbě vstupních parametrů modelu.

Někdy může být užitečné zobrazit korelace pomocí teplotní mapy.
Obzvláště pokud je korelační matice veliká, tak nám barvy mohou pomoci v orientaci.

In [ ]:
plt.figure(figsize = (10,8))
sns.heatmap(corr.abs(), annot=True, vmin=0, vmax=1)

Například sloupec CHAS (figurální proměnná Charles River) nemá žádnou souvislost s jinými prvky.

Naopak sloupce LSTAT, TAX, RAD, NOX, INDUS mají vztahy na další sloupce.

V příští hodině se bude snažit vytvořit statistický model, který na základě vstupních parametrů bude odhadovat cenu nemovitost MEDV.

Budeme k tomu používat lineární regresi.

Když se zaměříme na řádek MEDV, tak vhodnými vstupními parametry mohou být sloupce RM, LSTAT.

## Úprava a standardizace dat

### Vyčištění dat

Některé sloupce obsahují data NULL. Musíme se rozhodnout, jak tento problém vyřešit. 
* Neúplné řádky lze ze souboru dat odstranit.
* Problematické sloupce by neměly být vstupními parametry modelu.
* Záznamy s extrémní hodnoty můžeme z datového setu vyloučit. Například protože se jedná o chyby měření.

In [ ]:
print (data.isnull().sum())

In [ ]:
data=data.dropna()

Někdy je užitečné vyřadit data s extrémními hodnotami. 
Z datového souboru odstraníme řádky, kde je mediánová hodnota domu vyšší než 50.

In [ ]:
data = data[~(data['MEDV'] >= 40.0)]

### Standardizace dat

Každá funkce má jinou střední hodnotu a směrodatná odchylku.

Před vložením dat do matematického modelu bývá dobrým zvykem provést standardizaci.

Důvody:
* zabránit tomu, aby některé proměnné modelu dominovaly
* může pomoci rychleji konvergovat modely strojového učení
* může usnadnit interpretaci koeficientů modelu strojového učení

Výpočet:
* x_new = (x – střední hodnota) / směrodatná_odchylka
* průměr = součet (x) / počet (x)
* směrodatná_odchylka = sqrt( součet ( (x – průměr)^2 ) / počet (x))

Standardizaci můžeme dělat ručně. Vypočítáme si průměr a odchylku a data upravíme. 

In [ ]:
data["AGE"].mean()

In [ ]:
data["AGE"].std()

In [ ]:
data['AGE_STD'] = (data['AGE'] - data['AGE'].mean()) / data['AGE'].std()
data['LSTAT_STD'] = (data['LSTAT'] - data['LSTAT'].mean()) / data['LSTAT'].std()

Můžeme se podívat na distribuční grafy, jak se původní data změnila na nová data.

Tvar grafu je totožný, ale standardizovaný graf je relativně centrovaný okolo hodnoty 0.

In [ ]:
fig = plt.figure (figsize=(10, 5))
axes = fig.subplots (1, 2)           
ax1 = axes[0]
ax2 = axes[1]
sns.histplot(data['AGE'],ax=ax1, kde=True)
sns.histplot(data['AGE_STD'],ax=ax2, kde=True)

Totéž pro LSTAT

In [ ]:
fig = plt.figure (figsize=(10, 5))
axes = fig.subplots (1, 2)           
ax1 = axes[0]
ax2 = axes[1]
sns.histplot(data['LSTAT'],ax=ax1, kde=True)
sns.histplot(data['LSTAT_STD'],ax=ax2, kde=True)